# Metaflow with Tempo Example

We will train two models and deploy them with tempo within a Metaflow pipeline. To understand the core example see [here](https://tempo.readthedocs.io/en/latest/examples/multi-model/README.html)

![archtecture](architecture.png)

## MetaFlow Prequisites


### Install metaflow locally

```
pip install metaflow
```

### Setup AWS Metaflow Support

Note at present this is required even for a local run as artifacts are stored on S3.

[Install Metaflow with remote AWS support](https://docs.metaflow.org/metaflow-on-aws/metaflow-on-aws).

### Setup Conda-Forge Support

The flow will use conda-forge so you need to add that channel to conda.

```
conda config --add channels conda-forge
```



## Tempo Requirements

For deploying to a remote Kubernetes cluster with Seldon Core installed do the following steps:

### GKE Cluster

Create a GKE cluster and install Seldon Core on it using [Ansible to install Seldon Core on a Kubernetes cluster](https://github.com/SeldonIO/ansible-k8s-collection).

For Metaflow to connect to your GKE cluster from AWS it will need to authenticate inside the running pipeline. For this you will need to do some further steps as follows:

You will need to create two files in the flow src folder:

```bash
kubeconfig.yaml
gsa-key.json
```

Follow the steps outlined in [GKE server authentication](https://cloud.google.com/kubernetes-engine/docs/how-to/api-server-authentication#environments-without-gcloud).



## Iris Flow Summary

In [23]:
!python src/irisflow.py --environment=conda show

Metaflow 2.3.2 executing IrisFlow for user:clive

A Flow to train two Iris dataset models and combine them for inference with Tempo

The flow performs the following steps:

1) Load Iris Data
2) Train SKLearn LR Model
3) Train XGBoost LR Model
4) Create and deploy Tempo artifacts

Step start
    Download Iris classification datatset
    => train_sklearn, train_xgboost

Step train_sklearn
    Train a SKLearn Logistic Regression Classifier on dataset and save model as artifact
    => join

Step train_xgboost
    Train an XGBoost classifier on the dataset and save model as artifact
    => join

Step join
    Merge two training runs.
    => tempo

Step tempo
    Create Tempo artifacts locally and saved to S3 within the workflow bucket.
    Then either deploy locally to Docker or deploy to a remote Kubernetes cluster based on the
    --tempo-on-docker parameter
    => end

Step end
    End flow.



## Run Flow locally to deploy to Docker

To run the workflow with a local Docker deployment use the flag:

```
--tempo-on-docker true
```


In [32]:
!python src/irisflow.py \
    --environment=conda \
    run --tempo-on-docker true

Metaflow 2.3.2 executing IrisFlow for user:clive
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint not found, so extra checks are disabled.
Bootstrapping conda environment...(this could take a few minutes)
Including file src/conda.yaml of size 115B 
File persisted at s3://metaflow1-metaflows3bucket-dq2lr7x0v0nz/metaflow/data/IrisFlow/a94f1cff7702ed70807d16917bb282f51a28511e
Including file src/gsa-key.json of size 2KB 
File persisted at s3://metaflow1-metaflows3bucket-dq2lr7x0v0nz/metaflow/data/IrisFlow/6a24eaae697b14c2d8cdc2dcee1fce38b23b6ce0
Including file src/kubeconfig.yaml of size 2KB 
File persisted at s3://metaflow1-metaflows3bucket-dq2lr7x0v0nz/metaflow/data/IrisFlow/00e21f5423ea21f695831e970aa81a283ac25812
2021-08-09 16:26:11.721 Workflow starting (run-id 33):
2021-08-09 16:26:13.511 [33/start/179 (pid 3789808)] Task is starting.
2021-08-09 16:26:17.912 [33/start/179 (pid 3789808)] Task finished successfully.
2021-08-09 16:26:19.818 [33/train_sklear

2021-08-09 15:27:53.990 [33/tempo/183 (pid 3791157)] Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
2021-08-09 15:27:53.990 [33/tempo/183 (pid 3791157)] Collecting cachetools<5.0,>=2.0.0
2021-08-09 15:27:53.991 [33/tempo/183 (pid 3791157)] Using cached cachetools-4.2.2-py3-none-any.whl (11 kB)
2021-08-09 15:27:53.991 [33/tempo/183 (pid 3791157)] Collecting pyasn1-modules>=0.2.1
2021-08-09 15:27:53.991 [33/tempo/183 (pid 3791157)] Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
2021-08-09 15:27:53.991 [33/tempo/183 (pid 3791157)] Collecting rsa<5,>=3.1.4
2021-08-09 15:27:53.991 [33/tempo/183 (pid 3791157)] Using cached rsa-4.7.2-py3-none-any.whl (34 kB)
2021-08-09 15:27:53.999 [33/tempo/183 (pid 3791157)] Collecting pyasn1<0.5.0,>=0.4.6
2021-08-09 15:27:53.999 [33/tempo/183 (pid 3791157)] Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
2021-08-09 15:27:53.999 [33/tempo/183 (pid 3791157)] Collecting pyparsing>=2.0.2
2021-08-09 15:27:53.999 [33/te

2021-08-09 15:28:24.451 [33/tempo/183 (pid 3791157)] Preparing transaction: ...working... done
2021-08-09 15:28:24.593 [33/tempo/183 (pid 3791157)] Verifying transaction: ...working... done
2021-08-09 15:28:24.756 [33/tempo/183 (pid 3791157)] Executing transaction: ...working... done
2021-08-09 15:28:59.804 [33/tempo/183 (pid 3791157)] Remove all packages in environment /home/clive/anaconda3/envs/tempo-e150ca0c-c19d-4611-96c4-37624926e8ab:
2021-08-09 15:28:59.804 [33/tempo/183 (pid 3791157)] 
2021-08-09 15:28:59.804 [33/tempo/183 (pid 3791157)] Insights Manager not initialised as empty URL provided.
2021-08-09 15:29:09.922 [33/tempo/183 (pid 3791157)] {'output0': array([[0.00847207, 0.03168793, 0.95984   ]], dtype=float32), 'output1': 'xgboost prediction'}
2021-08-09 16:29:12.428 [33/tempo/183 (pid 3791157)] Task finished successfully.
2021-08-09 16:29:14.614 [33/end/184 (pid 3799779)] Task is starting.
2021-08-09 16:29:18.447 [33/end/184 (pid 3799779)] Task finished successfully.
2021

## Make Predictions with Metaflow Tempo Artifact

In [36]:
from metaflow import Flow
run = Flow('IrisFlow').latest_run

In [37]:
client = run.data.client_model

In [38]:
import numpy as np
client.predict(np.array([[1, 2, 3, 4]]))

{'output0': array([[0.00847207, 0.03168793, 0.95984   ]], dtype=float32),
 'output1': 'xgboost prediction'}

## Run Flow on AWS and Deploy to Remote Kubernetes

We will now run our flow on AWS Batch and will launch Tempo artifacts onto a remote Kubernetes cluster. make sure you have setup the cluster and have the appropriate authentication files as outlined above.

## Setup RBAC and Secret on Kubernetes Cluster

In [ ]:
!kubectl create ns production

In [ ]:
!kubectl create -f k8s/tempo-pipeline-rbac.yaml -n production

Create a Secret from the `k8s/s3_secret.yaml.tmpl` file by adding your AWS Key that can read from S3 and saving as `k8s/s3_secret.yaml`

In [ ]:
!kubectl create -f k8s/s3_secret.yaml -n production

## Run Metaflow on AWS Batch

In [6]:
!python src/irisflow.py \
    --environment=conda \
    --with batch:image=seldonio/seldon-core-s2i-python37-ubi8:1.10.0-dev \
    run

Metaflow 2.3.2 executing IrisFlow for user:clive
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint not found, so extra checks are disabled.
Bootstrapping conda environment...(this could take a few minutes)
Including file src/conda.yaml of size 115B 
File persisted at s3://metaflow1-metaflows3bucket-dq2lr7x0v0nz/metaflow/data/IrisFlow/a94f1cff7702ed70807d16917bb282f51a28511e
Including file src/gsa-key.json of size 2KB 
File persisted at s3://metaflow1-metaflows3bucket-dq2lr7x0v0nz/metaflow/data/IrisFlow/6a24eaae697b14c2d8cdc2dcee1fce38b23b6ce0
Including file src/kubeconfig.yaml of size 2KB 
File persisted at s3://metaflow1-metaflows3bucket-dq2lr7x0v0nz/metaflow/data/IrisFlow/00e21f5423ea21f695831e970aa81a283ac25812
2021-08-08 15:33:21.881 Workflow starting (run-id 32):
2021-08-08 15:33:23.615 [32/start/172 (pid 2469999)] Task is starting.
2021-08-08 14:33:24.695 [32/start/172 (pid 2469999)] [aa141922-797f-4e1e-8216-49b77312a969] Task is starting (status SUBM

2021-08-08 14:35:52.108 [32/join/175 (pid 2470870)] [36331d73-c958-4aa7-92a9-c0f3b6258d47] Downloading code package...
2021-08-08 14:35:52.612 [32/join/175 (pid 2470870)] [36331d73-c958-4aa7-92a9-c0f3b6258d47] Code package downloaded.
2021-08-08 14:35:52.622 [32/join/175 (pid 2470870)] [36331d73-c958-4aa7-92a9-c0f3b6258d47] Task is starting.
2021-08-08 14:35:52.987 [32/join/175 (pid 2470870)] [36331d73-c958-4aa7-92a9-c0f3b6258d47] Bootstrapping environment...
2021-08-08 14:36:09.321 [32/join/175 (pid 2470870)] [36331d73-c958-4aa7-92a9-c0f3b6258d47] Environment bootstrapped.
2021-08-08 14:36:18.563 [32/join/175 (pid 2470870)] [36331d73-c958-4aa7-92a9-c0f3b6258d47] Task finished with exit code 0.
2021-08-08 15:36:19.404 [32/join/175 (pid 2470870)] Task finished successfully.
2021-08-08 15:36:21.203 [32/tempo/176 (pid 2471152)] Task is starting.
2021-08-08 14:36:22.364 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] Task is starting (status SUBMITTED)...
2021-08-08 14:

2021-08-08 14:37:51.091 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] Ran pip subprocess with arguments:
2021-08-08 14:37:51.091 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] ['/opt/conda/envs/tempo-3b5a29db-9160-4897-89c8-3e5a250f8186/bin/python', '-m', 'pip', 'install', '-U', '-r', '/tmp/condaenv.idvat6xa.requirements.txt']
2021-08-08 14:37:51.091 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] Pip subprocess output:
2021-08-08 14:37:51.091 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] Collecting mlops-tempo
2021-08-08 14:37:51.091 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb]   Downloading mlops_tempo-0.3.0-py3-none-any.whl (74 kB)
2021-08-08 14:37:51.091 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] Collecting mlserver==0.3.2
2021-08-08 14:37:51.091 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb]   Downloading mlserver-0.3.2-py3-none-

2021-08-08 14:37:51.094 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb]   Using cached h11-0.12.0-py3-none-any.whl (54 kB)
2021-08-08 14:37:51.094 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] Collecting asgiref>=3.3.4
2021-08-08 14:37:51.094 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb]   Using cached asgiref-3.4.1-py3-none-any.whl (25 kB)
2021-08-08 14:37:51.095 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] Installing collected packages: zipp, typing-extensions, pycparser, urllib3, pyasn1, importlib-metadata, idna, charset-normalizer, cffi, starlette, six, rsa, requests, pydantic, pyasn1-modules, oauthlib, multidict, h11, cryptography, click, cachetools, asgiref, yarl, websocket-client, uvicorn, requests-oauthlib, pyyaml, python-dateutil, pyparsing, protobuf, orjson, numpy, grpcio, google-auth, fastapi, chardet, Authlib, attrs, async-timeout, seldon-deploy-sdk, redis, python-rclone, packaging, mlser

2021-08-08 14:38:03.270 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] Remove all packages in environment /opt/conda/envs/tempo-3b5a29db-9160-4897-89c8-3e5a250f8186:
2021-08-08 14:38:03.270 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb]
2021-08-08 14:38:08.608 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] Insights Manager not initialised as empty URL provided.
2021-08-08 14:38:18.854 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] {'output0': array([[0.00847207, 0.03168793, 0.95984   ]], dtype=float32), 'output1': 'xgboost prediction'}
2021-08-08 14:38:23.019 [32/tempo/176 (pid 2471152)] [71cbd837-9cf1-43a8-bdb9-e2c898f4f3eb] Task finished with exit code 0.
2021-08-08 15:38:23.844 [32/tempo/176 (pid 2471152)] Task finished successfully.
2021-08-08 15:38:24.892 [32/end/177 (pid 2471980)] Task is starting.
2021-08-08 14:38:25.925 [32/end/177 (pid 2471980)] [d6392d28-0aec-416f-ac0e-670e0ef63c0b] Task is st

## Make Predictions with Metaflow Tempo Artifact

In [7]:
from metaflow import Flow
run = Flow('IrisFlow').latest_run

In [8]:
client = run.data.client_model

In [9]:
import numpy as np
client.predict(np.array([[1, 2, 3, 4]]))

{'output0': array([[0.00847207, 0.03168793, 0.95984   ]], dtype=float32),
 'output1': 'xgboost prediction'}